# Predicting the simultaneous effect of multiple mutations

## Summary

While `ELASPIC2` was not explicitly trained to predict the effect of multiple mutations, we believe that reasonable accuracy can be achieved by using the following strategy. Say we wish to mutate protein `ADEK` to protein `GEDR`. We could instead evaluate the effect of the following four mutations, and then combine the results:

1. `ADEK` → `GDEK`
1. `GDEK` → `GEEK`
1. `GEEK` → `GEDK`
1. `GEDK` → `GEDR`

Furthermore, both `ProtBert` and `ProteinSolver` can be natively used to obtain a probability score for the wildtype and the mutant sequence, and the difference between those probabilities should correlate with the stability of the protein [1].

This notebook demonstrates how to use both of the strategies described above to calculate the change in stability between a wildtype and a mutant sequence that differ by more than one amino acid.

[1]: Strokach A, Becerra D, Corbi-Verge C, Perez-Riba A, Kim PM. _Fast and flexible protein design using deep graph neural networks_. Cell Systems (2020); 11: 1–10. doi: [10.1016/j.cels.2020.08.016](https://www.cell.com/cell-systems/fulltext/S2405-4712(20)30327-6)

## Google colab

In [1]:
try:
    import google.colab
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False
    
GOOGLE_COLAB

True

In [2]:
if GOOGLE_COLAB:
    !pip install "torch==1.8.0";
    !pip install -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html --default-timeout=600 \
        "transformers==3.3.1" \
        "torch-scatter==2.0.6" \
        "torch-sparse==0.6.9" \
        "torch-cluster==1.5.9" \
        "torch-spline-conv==1.2.1" \
        "torch-geometric==1.6.1" \
        "https://gitlab.com/kimlab/kmbio/-/archive/v2.1.0/kmbio-v2.1.0.zip" \
        "https://gitlab.com/kimlab/kmtools/-/archive/v0.2.8/kmtools-v0.2.8.zip" \
        "https://gitlab.com/ostrokach/proteinsolver/-/archive/v0.1.25/proteinsolver-v0.1.25.zip" \
        "git+https://gitlab.com/elaspic/elaspic2.git"

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
  Using cached https://gitlab.com/kimlab/kmbio/-/archive/v2.1.0/kmbio-v2.1.0.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://gitlab.com/kimlab/kmtools/-/archive/v0.2.8/kmtools-v0.2.8.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://gitlab.com/ostrokach/proteinsolver/-/archive/v0.1.25/proteinsolver-v0.1.25.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://gitlab.com/elaspic/elaspic2.git to /tmp/pip-req-build-zabhr9ft
  Running command git clone -q https://gitlab.com/elaspic/elaspic2.git /tmp/pip-req-build-zabhr9ft
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Prepari

## Imports

In [3]:
import io
import tempfile
from copy import deepcopy
from pathlib import Path

import elaspic2 as el2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import proteinsolver
import requests
import torch
from kmbio import PDB
from kmtools import structure_tools
from scipy import stats
from tqdm.notebook import tqdm

## Parameters

In [4]:
NOTEBOOK_DIR = Path("10_multiresidue_demo").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/content/10_multiresidue_demo')

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

## Helper functions

In [6]:
def structure_to_blob(structure):
    with tempfile.NamedTemporaryFile(suffix=".pdb") as out:
        PDB.save(structure, out.name)
        with open(out.name, "rt") as fin:
            data = fin.read()
    return data

In [7]:
def sequence_matches_structure(sequence, structure_blob):
    with tempfile.NamedTemporaryFile(suffix=".pdb") as tmp_file:
        with open(tmp_file.name, "wt") as fout:
            fout.write(structure_blob)
        structure = PDB.load(tmp_file.name)

    chain_sequence = structure_tools.get_chain_sequence(
        structure[0]["A"], if_unknown="replace", unknown_residue_marker=""
    )
    return sequence == chain_sequence

In [8]:
def sequence_pair_to_mutations(sequence_wt, sequence_mut):
    assert len(sequence_wt) == len(sequence_mut)

    mutations = []
    for idx, (aa_wt, aa_mut) in enumerate(zip(sequence_wt, sequence_mut)):
        if aa_wt != aa_mut:
            mutations.append(f"{aa_wt}{idx + 1}{aa_mut}")
    return mutations

In [9]:
def apply_mutations(protein_features, mutation):
    protein_features = deepcopy(protein_features)

    for mutation in mutations:
        protein_features = apply_mutation(protein_features, mutation)

    return protein_features


def apply_mutation(protein_features, mutation):
    """

    Warning: This function mutates `protein_features`!
    """
    aa_wt = mutation[0]
    pos = int(mutation[1:-1])
    aa_mut = mutation[-1]

    # Mutate ProtBert features
    aa_list = list(protein_features.protbert_data.sequence)
    assert aa_list[pos - 1] == aa_wt
    aa_list[pos - 1] = aa_mut

    protein_features = protein_features._replace(
        protbert_data = protein_features.protbert_data._replace(sequence="".join(aa_list))
    )
    
    # Mutate ProteinSolver features
    assert (protein_features.proteinsolver_data.x[pos - 1] == proteinsolver.utils.AMINO_ACIDS.index(aa_wt)).item()
    protein_features.proteinsolver_data.x[pos - 1] = proteinsolver.utils.AMINO_ACIDS.index(aa_mut)

    return protein_features

## Workspace

### Load protein structure

In [10]:
structure_ref = PDB.load("rcsb://6ZOY.pdb")

In [11]:
PDB.view_structure(structure_ref)

NGLWidget()

In [12]:
structure = structure_tools.extract_domain(structure_ref, [structure_tools.DomainDef(0, "A", 260, 480)])

In [13]:
PDB.view_structure(structure)

NGLWidget()

### Get wildtype and mutant chain sequences

In [14]:
protein_sequence_wt = structure_tools.get_chain_sequence(
    structure[0]["A"], if_unknown="replace", unknown_residue_marker=""
)

protein_sequence_wt

'QPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVCPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNF'

In [15]:
protein_sequence_mut = """\
WWWWWIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVCPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSN\
NLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNF\
"""

protein_sequence_mut

'WWWWWIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVCPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNF'

In [16]:
mutations = sequence_pair_to_mutations(protein_sequence_wt, protein_sequence_mut)

mutations

['Q1W', 'P2W', 'T3W', 'E4W', 'S5W']

### Instantiate the `ELASPIC2` model and featurize protein



In [17]:
model = el2.ELASPIC2(device=device)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [18]:
structure_file_obj = tempfile.NamedTemporaryFile(suffix=".pdb")
PDB.save(structure, structure_file_obj.name)

In [19]:
protein_features_wt = model.build(
    structure_file=structure_file_obj.name,
    protein_sequence=protein_sequence_wt,
    ligand_sequence=None,
    remove_hetatms=True,
)

protein_features_wt

ELASPIC2Data(is_interface=False, protbert_data=ProtBertData(sequence='QPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVCPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNF'), proteinsolver_data=Data(edge_attr=[9590, 2], edge_index=[2, 9590], x=[221]))

### Calculate aggregate EL2 score for all mutations

In [20]:
results = []

protein_features_tmp = deepcopy(protein_features_wt)
for mutation in tqdm(mutations):
    mutation_features = model.analyze_mutation(mutation, protein_features_tmp)
    stability_pred = model.predict_mutation_effect([mutation_features])
    results.append({
        "mutation": mutation,
        "protbert_score": mutation_features["protbert_core_score_mut"] - mutation_features["protbert_core_score_wt"],
        "proteinsolver_score": mutation_features["proteinsolver_core_score_mut"] - mutation_features["proteinsolver_core_score_wt"],
        "el2_score": stability_pred[0].item(),
    })
    protein_features_tmp = apply_mutation(protein_features_tmp, mutation)

In [21]:
results_df = pd.DataFrame(results)

results_df

,mutation,protbert_score,proteinsolver_score,el2_score
0,Q1W,-0.019935,-0.025641,0.454411
1,P2W,-0.025096,-0.035472,0.678083
2,T3W,-0.047077,-0.067026,0.494758
3,E4W,-0.012974,-0.048199,1.452932
4,S5W,-0.003738,-0.117139,1.549611


In [22]:
protbert_score = results_df["protbert_score"].sum()

protbert_score  # -0.10881942184641957 (mutant is less probable)

-0.10881942184641957

In [23]:
proteinsolver_score = results_df["proteinsolver_score"].sum()

proteinsolver_score # -0.29347743903053924 (mutant is less probable)

-0.29347743903053924

In [24]:
total_el2_score = results_df["el2_score"].sum()

total_el2_score # 4.629795396393689 (ΔΔG is positive; mutation is destabilizing)

4.629795396393689

### Calculate global difference in ProtBert and ProtienSolver scores

In [25]:
protein_features_mut = apply_mutations(protein_features_wt, mutations)

results_global = []
for idx, (aa_wt, aa_mut) in tqdm(enumerate(zip(protein_sequence_wt, protein_sequence_mut)), total=len(protein_sequence_wt)):
    # Calculate probability of a residue in the context of the wildtype structure
    mutation_wt2any = f"{aa_wt}{idx + 1}{'A' if aa_wt != 'A' else 'G'}"
    mutation_features_wt = model.analyze_mutation(mutation_wt2any, protein_features_wt)

    # Calculate probability of a residue in the context of the mutant structure
    mutation_mut2any = f"{aa_mut}{idx + 1}{'A' if aa_mut != 'A' else 'G'}"
    mutation_features_mut = model.analyze_mutation(mutation_mut2any, protein_features_mut)

    results_global.append({
        "position": idx + 1,
        "aa_wt": aa_wt,
        "aa_mut": aa_mut,
        "protbert_score_wt": mutation_features_wt["protbert_core_score_wt"],
        "proteinsolver_score_wt": mutation_features_wt["proteinsolver_core_score_wt"],
        "protbert_score_mut": mutation_features_mut["protbert_core_score_wt"],
        "proteinsolver_score_mut": mutation_features_mut["proteinsolver_core_score_wt"],
    })

In [26]:
results_global_df = pd.DataFrame(results_global)

results_global_df

,position,aa_wt,aa_mut,protbert_score_wt,proteinsolver_score_wt,protbert_score_mut,proteinsolver_score_mut
0,1,Q,W,0.026342,0.028866,0.037111,0.003031
1,2,P,W,0.044389,0.043030,0.078061,0.004068
2,3,T,W,0.071309,0.078954,0.080384,0.003102
3,4,E,W,0.038644,0.047481,0.090949,0.000510
4,5,S,W,0.066574,0.114781,0.068657,0.002685
...,...,...,...,...,...,...,...
216,217,K,K,0.064456,0.107810,0.067017,0.082281
217,218,C,C,0.049940,0.003146,0.047689,0.005638
218,219,V,V,0.075891,0.363526,0.073260,0.237637
219,220,N,N,0.072539,0.103463,0.073284,0.052776


In [27]:
protbert_score = results_global_df["protbert_score_mut"].sum() - results_global_df["protbert_score_wt"].sum()

protbert_score  # -0.0995915224775672 (mutant is less probable)

-0.0995915224775672

In [28]:
proteinsolver_score = results_global_df["proteinsolver_score_mut"].sum() - results_global_df["proteinsolver_score_wt"].sum()

proteinsolver_score  # -0.7629658342921175 (mutant is less probable)

-0.7629658342921175